In [ ]:

import pandas as pd
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataset
df = pd.read_csv('/content/drive/MyDrive/Digified/final.csv')
df.drop(['index','Unnamed: 0'], axis=1, inplace=True)
df.drop_duplicates(inplace = True)
df

,Full_name,class
0,اعغضز أاكب امجد,F
1,اسداءخصبسحةءائض,F
2,اعتدضل فائز سويد,F
3,احمد عبدالءبار عامل,F
4,سعيده محرم قطب,T
...,...,...
5995,عقاب ضرام كلاب,T
5996,ذعار مطيع امير,T
5997,اشهب اكمل ناهل,T
5998,وق تطإط تاتل,F


In [ ]:
train_size = int(len(df) * .8)

print(int(len(df['Full_name'])))
print(train_size)

6000
4800


In [ ]:
## spliting the data
texts= df['Full_name']
tags = df['class']


train_posts = df['Full_name'][:train_size]
train_tags = df['class'][:train_size]



test_posts = df['Full_name'][train_size:]
test_tags =  df['class'][train_size:]

In [ ]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

In [ ]:
encoder = LabelEncoder()
encoder.fit(tags)
tagst=encoder.fit_transform(tags)

num_classes = int((len(set(tagst))))
print((len(set(tagst))))

y_train = encoder.fit_transform(train_tags)
y_test = encoder.fit_transform(test_tags)

2


In [ ]:
y_train= keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size

In [ ]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

In [ ]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              6878208   
                                                                 
 activation (Activation)     (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2050      
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
Total params: 6,880,258
Trainable params: 6,880,258
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 100
epochs = 2

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
#model.save('/content/drive/MyDrive/Digified/my_model.h5' )
print('Model Saved!')

Epoch 1/2
44/44 [==============================] - 7s 117ms/step - loss: 0.5352 - categorical_accuracy: 0.7660 - recall: 0.6863 - precision: 0.7575 - f1_metric: 0.7193 - true_positives: 2965.0000 - true_negatives: 3371.0000 - false_positives: 949.0000 - false_negatives: 1355.0000 - val_loss: 0.3000 - val_categorical_accuracy: 0.9500 - val_recall: 0.9271 - val_precision: 0.9549 - val_f1_metric: 0.9411 - val_true_positives: 445.0000 - val_true_negatives: 459.0000 - val_false_positives: 21.0000 - val_false_negatives: 35.0000
Epoch 2/2
44/44 [==============================] - 4s 95ms/step - loss: 0.0884 - categorical_accuracy: 0.9894 - recall: 0.9843 - precision: 0.9875 - f1_metric: 0.9847 - true_positives: 4252.0000 - true_negatives: 4266.0000 - false_positives: 54.0000 - false_negatives: 68.0000 - val_loss: 0.1595 - val_categorical_accuracy: 0.9542 - val_recall: 0.9458 - val_precision: 0.9598 - val_f1_metric: 0.9531 - val_true_positives: 454.0000 - val_true_negatives: 461.0000 - val_fals

In [ ]:
dependencies = {
    'f1_metric': f1_metric
}

model = keras.models.load_model('/content/drive/MyDrive/Digified/my_model.h5', custom_objects=dependencies)

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(x_test,y_test,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)

Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[0.2008522003889084, 0.9291666746139526, 0.9266666769981384, 0.9274395108222961, 0.9279749989509583, 1112.0, 1113.0, 87.0, 88.0]


In [ ]:
## test the model
for x in df["Full_name"][train_size:]:

    tokens = tokenizer.texts_to_matrix([x], mode='tfidf')
    predict_x=model.predict(np.array(tokens)) 
    c=np.argmax(predict_x,axis=1)
    xc = encoder.inverse_transform(c)


    print(x,"= \t",xc)
    print(predict_x)

1/1 [==============================] - 0s 315ms/step
نضير معتوق ذبيان = 	 ['T']
[[0.08292952 0.93538946]]
1/1 [==============================] - 0s 21ms/step
حبيمهؤءناعئكميتي = 	 ['F']
[[0.7806976  0.19487756]]
1/1 [==============================] - 0s 23ms/step
عبدي معد رهيف = 	 ['T']
[[0.20131902 0.80133677]]
1/1 [==============================] - 0s 21ms/step
نعمت زاخر نزار = 	 ['T']
[[0.07376396 0.93222976]]
1/1 [==============================] - 0s 29ms/step
نسرين مثاب مفتاح = 	 ['T']
[[0.11693105 0.8788642 ]]
1/1 [==============================] - 0s 26ms/step
مديؤؤتئتس غبدإؤريل = 	 ['F']
[[0.8242745  0.15841907]]
1/1 [==============================] - 0s 21ms/step
ساهدهءباسط كاأب = 	 ['F']
[[0.8456652  0.14010113]]
1/1 [==============================] - 0s 22ms/step
اهيم مالك عبدالمنعم = 	 ['T']
[[0.1411803 0.8678409]]
1/1 [==============================] - 0s 23ms/step
خليفج إعانهظكيدص = 	 ['F']
[[0.8053359  0.21845786]]
1/1 [==============================] - 0s 23ms/step
دانا 

In [ ]:
### test the model
def prediction(name):
  tokens = tokenizer.texts_to_matrix([name], mode='tfidf')
  predict_x=model.predict(np.array(tokens)) 
  c=np.argmax(predict_x,axis=1)
  xc = encoder.inverse_transform(c)
  if predict_x[0][1]> 0.50 :
    if predict_x[0][1]- predict_x[0][0]>0.001:
      Confidence="True Name with High Confidence"
    else:
      Confidence="True Name with Low Confidence"
  else:
    Confidence="False Name"
  return xc[0],Confidence,predict_x

In [ ]:
prediction('باسم وحيد السيد')

1/1 [==============================] - 0s 24ms/step


('T',
 'True Name with High Confidence',
 array([[0.5080282, 0.5115268]], dtype=float32))